In [1]:
import pygame
import random
import sys

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Initialize Pygame
pygame.init()


(5, 0)

In [3]:
# Constants
SCREEN_WIDTH = 400
SCREEN_HEIGHT = 600
GRAVITY = 0.5
JUMP_STRENGTH = -10
PIPE_WIDTH = 80
PIPE_GAP = 200
PIPE_SPEED = 3

In [4]:
# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 100, 255)
YELLOW = (255, 255, 0)
RED = (255, 0, 0)

In [5]:
class Bird:
    def __init__(self):
        self.x = 80
        self.y = SCREEN_HEIGHT // 2
        self.velocity = 0
        self.radius = 20
        
    def jump(self):
        self.velocity = JUMP_STRENGTH
        
    def update(self):
        self.velocity += GRAVITY
        self.y += self.velocity
        
    def draw(self, screen):
        pygame.draw.circle(screen, YELLOW, (int(self.x), int(self.y)), self.radius)
        # Draw eye
        pygame.draw.circle(screen, BLACK, (int(self.x + 8), int(self.y - 5)), 3)
        # Draw beak
        pygame.draw.polygon(screen, RED, [(int(self.x + self.radius), int(self.y)), 
                                       (int(self.x + self.radius + 15), int(self.y - 5)),
                                       (int(self.x + self.radius + 15), int(self.y + 5))])
        
    def get_rect(self):
        return pygame.Rect(self.x - self.radius, self.y - self.radius, 
                          self.radius * 2, self.radius * 2)


In [6]:
class Pipe:
    def __init__(self, x):
        self.x = x
        self.gap_y = random.randint(150, SCREEN_HEIGHT - 150 - PIPE_GAP)
        self.passed = False
        
    def update(self):
        self.x -= PIPE_SPEED
        
    def draw(self, screen):
        # Top pipe
        pygame.draw.rect(screen, GREEN, 
                        (self.x, 0, PIPE_WIDTH, self.gap_y))
        # Bottom pipe
        pygame.draw.rect(screen, GREEN, 
                        (self.x, self.gap_y + PIPE_GAP, PIPE_WIDTH, 
                         SCREEN_HEIGHT - self.gap_y - PIPE_GAP))
        # Pipe caps
        pygame.draw.rect(screen, GREEN, 
                        (self.x - 5, self.gap_y - 20, PIPE_WIDTH + 10, 20))
        pygame.draw.rect(screen, GREEN, 
                        (self.x - 5, self.gap_y + PIPE_GAP, PIPE_WIDTH + 10, 20))
        
    def get_rects(self):
        top_rect = pygame.Rect(self.x, 0, PIPE_WIDTH, self.gap_y)
        bottom_rect = pygame.Rect(self.x, self.gap_y + PIPE_GAP, PIPE_WIDTH, 
                                 SCREEN_HEIGHT - self.gap_y - PIPE_GAP)
        return [top_rect, bottom_rect]
        
    def is_off_screen(self):
        return self.x + PIPE_WIDTH < 0

In [7]:
class Game:
    def __init__(self):
        self.screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
        pygame.display.set_caption("Flappy Bird")
        self.clock = pygame.time.Clock()
        self.font = pygame.font.Font(None, 36)
        self.big_font = pygame.font.Font(None, 72)
        self.reset_game()
        
    def reset_game(self):
        self.bird = Bird()
        self.pipes = []
        self.score = 0
        self.game_over = False
        self.game_started = False
        
    def handle_events(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    if not self.game_started:
                        self.game_started = True
                    elif self.game_over:
                        self.reset_game()
                        self.game_started = True
                    else:
                        self.bird.jump()
                elif event.key == pygame.K_ESCAPE:
                    return False
        return True
        
    def update(self):
        if not self.game_started or self.game_over:
            return
            
        self.bird.update()
        
        # Check if bird hits ground or ceiling
        if self.bird.y > SCREEN_HEIGHT - self.bird.radius or self.bird.y < self.bird.radius:
            self.game_over = True
            
        # Generate pipes
        if len(self.pipes) == 0 or self.pipes[-1].x < SCREEN_WIDTH - 200:
            self.pipes.append(Pipe(SCREEN_WIDTH))
            
        # Update pipes
        for pipe in self.pipes[:]:
            pipe.update()
            if pipe.is_off_screen():
                self.pipes.remove(pipe)
                
        # Check collisions
        bird_rect = self.bird.get_rect()
        for pipe in self.pipes:
            for pipe_rect in pipe.get_rects():
                if bird_rect.colliderect(pipe_rect):
                    self.game_over = True
                    
        # Check score
        for pipe in self.pipes:
            if not pipe.passed and pipe.x + PIPE_WIDTH < self.bird.x:
                pipe.passed = True
                self.score += 1
                
    def draw(self):
        # Sky gradient
        for y in range(SCREEN_HEIGHT):
            color_intensity = int(135 + (120 * y / SCREEN_HEIGHT))
            color = (135, 206, min(255, color_intensity))
            pygame.draw.line(self.screen, color, (0, y), (SCREEN_WIDTH, y))
            
        # Draw pipes
        for pipe in self.pipes:
            pipe.draw(self.screen)
            
        # Draw bird
        self.bird.draw(self.screen)
        
        # Draw score
        score_text = self.font.render(f"Score: {self.score}", True, WHITE)
        score_rect = score_text.get_rect()
        score_rect.topleft = (10, 10)
        pygame.draw.rect(self.screen, BLACK, score_rect.inflate(20, 10))
        self.screen.blit(score_text, (20, 15))
        
        # Draw game states
        if not self.game_started:
            title_text = self.big_font.render("FLAPPY BIRD", True, WHITE)
            start_text = self.font.render("Press SPACE to start", True, WHITE)
            title_rect = title_text.get_rect(center=(SCREEN_WIDTH//2, SCREEN_HEIGHT//2 - 50))
            start_rect = start_text.get_rect(center=(SCREEN_WIDTH//2, SCREEN_HEIGHT//2 + 20))
            
            pygame.draw.rect(self.screen, BLACK, title_rect.inflate(40, 20))
            pygame.draw.rect(self.screen, BLACK, start_rect.inflate(40, 20))
            self.screen.blit(title_text, title_rect)
            self.screen.blit(start_text, start_rect)
            
        elif self.game_over:
            game_over_text = self.big_font.render("GAME OVER", True, WHITE)
            restart_text = self.font.render("Press SPACE to restart", True, WHITE)
            final_score_text = self.font.render(f"Final Score: {self.score}", True, WHITE)
            
            game_over_rect = game_over_text.get_rect(center=(SCREEN_WIDTH//2, SCREEN_HEIGHT//2 - 50))
            restart_rect = restart_text.get_rect(center=(SCREEN_WIDTH//2, SCREEN_HEIGHT//2 + 20))
            final_score_rect = final_score_text.get_rect(center=(SCREEN_WIDTH//2, SCREEN_HEIGHT//2 - 10))
            
            pygame.draw.rect(self.screen, BLACK, game_over_rect.inflate(40, 20))
            pygame.draw.rect(self.screen, BLACK, restart_rect.inflate(40, 20))
            pygame.draw.rect(self.screen, BLACK, final_score_rect.inflate(40, 20))
            
            self.screen.blit(game_over_text, game_over_rect)
            self.screen.blit(final_score_text, final_score_rect)
            self.screen.blit(restart_text, restart_rect)
            
        # Draw instructions
        if self.game_started and not self.game_over:
            instruction_text = pygame.font.Font(None, 24).render("SPACE to flap, ESC to quit", True, WHITE)
            instruction_rect = instruction_text.get_rect()
            instruction_rect.bottomright = (SCREEN_WIDTH - 10, SCREEN_HEIGHT - 10)
            pygame.draw.rect(self.screen, BLACK, instruction_rect.inflate(20, 10))
            self.screen.blit(instruction_text, (instruction_rect.x + 10, instruction_rect.y + 5))
            
        pygame.display.flip()
        
    def run(self):
        running = True
        while running:
            running = self.handle_events()
            self.update()
            self.draw()
            self.clock.tick(60)
            
        pygame.quit()
        sys.exit()

if __name__ == "__main__":
    game = Game()
    game.run()

SystemExit: 

C:\Users\anas0\anaconda5\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
